In [348]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import StaleElementReferenceException, ElementNotInteractableException, NoSuchElementException
import time
import re
import json
from wakepy import keepawake
import pandas as pd

In [349]:
df = pd.DataFrame(columns=["Store", "Item Name", "Price", "Units", "Unit Price", "Amount", "Tags", "In Stock", "Image"])

In [350]:
# Create a new instance of the Chrome driver
# executable_path='C:/path/to/chromedriver.exe
# driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome(executable_path='/Users/jakesanghavi/Downloads/chromedriver-mac-arm64/chromedriver')
driver.maximize_window()

# Open SoundCloud
driver.get('https://www.instacart.com/aldi')

wait = WebDriverWait(driver, 5)  # Maximum wait time of 5 seconds

In [351]:
# Remove modals that block website
elements_to_delete = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'ReactModalPortal')))

for element in elements_to_delete:
    driver.execute_script("arguments[0].remove();", element)

# This allows us to scroll and click again
body_element = driver.find_element_by_tag_name('body')
driver.execute_script("arguments[0].style.overflow = 'visible';", body_element)

In [352]:
# Function to scroll slowly to the bottom
def slow_scroll_to_bottom():
    actions = ActionChains(driver)
    t = 0
    while t < 10:
        actions.send_keys(Keys.PAGE_DOWN).perform()
        # Adjust the sleep time based on your needs
        time.sleep(1)
        # Check if you've reached the bottom of the page
        if driver.execute_script("return (window.innerHeight + window.scrollY) >= document.body.scrollHeight;"):
            break
        
        t += 1

# Scroll slowly to the bottom
slow_scroll_to_bottom()

time.sleep(2)

# Find all elements with aria-label="item carousel"
items = driver.find_elements(By.CSS_SELECTOR, '[aria-label="item carousel"]')

In [353]:
j = 1

reakit_portals = driver.find_elements(By.CLASS_NAME, '__reakit-portal')

to_concat = []

# Iterate through each item
for item in items:
    # Get the first div child
    first_div_child = item.find_element(By.CSS_SELECTOR, 'div:nth-child(1)')

    # Go into the second div within the first div child
    second_div = first_div_child.find_element(By.CSS_SELECTOR, 'div:nth-child(2)')
    
    ActionChains(driver).move_to_element(second_div).perform()

    # Click the first button within the second div
    first_button = second_div.find_element(By.CSS_SELECTOR, 'button:nth-child(1)')
    first_button.click()
    
    modal = reakit_portals[j].find_element(By.CSS_SELECTOR, 'div:nth-child(1) div:nth-child(1) div:nth-child(2)')
        
#     ActionChains(driver).move_to_element(modal).send_keys(Keys.END).perform()
    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)
    time.sleep(1)
    driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight);", modal)

    # Careful here: may not load all products. We may have to add slower scrolling
    product_list = modal.find_elements(By.CSS_SELECTOR, 'div:nth-child(1) ul li')
    
    for product in product_list:
#         img_tag = product.find_element(By.XPATH, './/img')

        text_list = []
        unit_price = ""
        individual_weight = ""
        src_value = ""
        stock = ""

        name = product.find_element(By.CSS_SELECTOR, 'div div div a').get_attribute('href')
        
        try:
            img_tag = product.find_element(By.CSS_SELECTOR, 'img')
            src_value = img_tag.get_attribute('srcset')
        except NoSuchElementException:

            continue
        
        try:
            text_data = product.find_elements(By.CSS_SELECTOR, 'span:not(:has(span))')
    
            for span in text_data:
                html = span.text
                if len(html) > 2:
                    text_list.append(html)                    
                        
            try:
                unit_price = product.find_element(By.CSS_SELECTOR, 'div[title]').text
            except NoSuchElementException:
                unit_price = ""
              
            k = 4
            
            while k > 0:
                try:
                    # Try to find the element with the specified CSS selector
                    individual_weight_element = product.find_element(By.CSS_SELECTOR, f'div[aria-label="Product"] a div:nth-child(2) div:nth-child({k})')

                    if 'stock' in individual_weight_element.text:
                        stock = individual_weight_element.text
                        k -= 1
                        continue


                    else:
                        individual_weight = individual_weight_element.text
                        k = 0
                except NoSuchElementException:
                    # If not found, set individual_weight to an empty string
                    individual_weight = ""
                    k -= 1
            
        except NoSuchElementException:
            continue
              
        tags = []
        price = ""
        name = ""
        units = ""

        src = src_value.split()
        if len(src) != 0:
            src = src[0]
        else:
            src = ""

        text_list = [element for element in text_list if element.strip() != ""]


        price_index = next(i for i, element in enumerate(text_list) if element.startswith("$"))
        try:
            units_index = next(i for i, element in enumerate(text_list) if element.startswith("/") or element.startswith("each"))
            units = text_list.pop(units_index)
        except StopIteration:
            units_index = None 

        for i in range(len(text_list)):
            if i < price_index:
                tags.append(text_list[i])
            elif i == price_index:
                price = text_list[i]
            elif i == price_index + 1:
                name = text_list[i]

        cols = df.columns
        row = pd.DataFrame(data=[["Aldi", name, price, units, unit_price, individual_weight, tags, stock, src]], columns=cols)

    #     df = pd.concat([df, row], ignore_index=True)
        to_concat.append(row)
            
    print(j)
    i += 1
    
# Process
## In the text_data, remove data like 'starts with each' or 'Add'.
## Tags are whatever come before $
## Price is whatever starts with $
# Item name is whatever comes right after $
# Amount or unit price is whatever comes right after Name
# Make sure to ignore empty strings

1


ElementClickInterceptedException: Message: element click intercepted: Element <button type="button" aria-haspopup="dialog" class="e-1bfzg3z">...</button> is not clickable at point (1240, 556). Other element would receive the click: <div class="e-svklq">...</div>
  (Session info: chrome=121.0.6167.139)


In [ ]:
driver.quit()

In [354]:
df = pd.concat([df] + to_concat, ignore_index=True)
mask = df['Unit Price'] == df['Amount']
df.loc[mask, 'Unit Price'] = ""
df

,Store,Item Name,Price,Units,Unit Price,Amount,Tags,In Stock,Image
0,Aldi,Bananas,$0.18,each (est.),$0.49 / lb,About 0.37 lb each,[],,https://www.instacart.com/image-server/257x257...
1,Aldi,"Strawberries, Package",$3.25,,,1 lb,[],,https://www.instacart.com/image-server/257x257...
2,Aldi,Blueberries Package,$3.55,,,1 pint container,[],Many in stock,https://www.instacart.com/image-server/257x257...
3,Aldi,"Mandarins, Bag",$3.09,,,3 lb,[],Many in stock,https://www.instacart.com/image-server/257x257...
4,Aldi,"Mission Mini Avocados, Bag",$2.55,,,6 ct,[],,https://www.instacart.com/image-server/257x257...
5,Aldi,Lemons,$4.19,,,2 lb bag,[],Many in stock,https://www.instacart.com/image-server/257x257...
6,Aldi,Raspberries Package,$2.75,,,6 oz Container,[],,https://www.instacart.com/image-server/257x257...
7,Aldi,Blackberries,$2.45,,,6 oz container,[],Many in stock,https://www.instacart.com/image-server/257x257...
8,Aldi,Red Grapes,$4.40,/pkg (est.),$2.19 / lb,About 2.01 lb / package,[],,https://www.instacart.com/image-server/257x257...
9,Aldi,"Limes, Bag",$2.65,,,1 lb,[],Many in stock,https://www.instacart.com/image-server/257x257...


In [355]:
df.to_csv("instacart_scraper_v1_results.csv")